# Querying the data

Your boss is excited that you now have the schema for the database and all the sales data is in one location. Since you've done such a great job they would like you to get some up-to-date metrics from the data. The business can then start making more data-driven decisions and get a better understanding of its sales. In this milestone, you will be tasked with answering business questions and extracting the data from the database using SQL.

In [4]:
from classes.database_utils import DatabaseConnector
from decouple import AutoConfig
import pandas as pd

config = AutoConfig(' ')

if __name__ == "__main__":
    file_path = config('local_db_path')
    db_connector = DatabaseConnector(file_path)
    engine = db_connector._init_db_engine()

## Task 1: How many stores does the business have and in which countries? 

The Operations team would like to know which countries we currently operate in and which country now has the most stores. 
Perform a query on the database to get the information, it should return the following information:

| country  | total_no_stores |
| -------  | --------------- |
| GB       |             265 |
| DE       |             141 |
| US       |              34 |


In [5]:
df = pd.read_sql_query("""
    SELECT country_code AS country, COUNT(*) AS total_no_stores
    FROM dim_store_details
    WHERE address NOT IN ('N/A')
    GROUP BY country_code
    ORDER BY total_no_stores DESC;
""", con=engine)
display(df)

,country,total_no_stores
0,GB,265
1,DE,141
2,US,34


## Task 2: Which locations currently have the most stores?

The business stakeholders would like to know which locations currently have the most stores.
They would like to close some stores before opening more in other locations.
Find out which locations have the most stores currently. The query should return the following:
|     locality      | total_no_stores |
|-------------------|-----------------|
| Chapletown        |              14 |
| Belper            |              13 |
| Bushley           |              12 |
| Exeter            |              11 |
| High Wycombe      |              10 |
| Arbroath          |              10 |
| Rutherglen        |              10 |

In [6]:
df = pd.read_sql_query("""
    SELECT locality, COUNT(*) AS total_no_stores
    FROM dim_store_details
    WHERE address NOT IN ('N/A')
    GROUP BY locality
    ORDER BY total_no_stores DESC
    LIMIT 7;
""", con=engine)
display(df)

,locality,total_no_stores
0,Chapletown,14
1,Belper,13
2,Bushey,12
3,Exeter,11
4,Arbroath,10
5,High Wycombe,10
6,Rutherglen,10


## Task 3: Which months produced the largest amount of sales?

Query the database to find out which months have produced the most sales. The query should return the following information:
| total_sales | month |
|-------------|-------|
|   673295.68 |     8 |
|   668041.45 |     1 |
|   657335.84 |    10 |
|   650321.43 |     5 |
|   645741.70 |     7 |
|   645463.00 |     3 |

In [7]:
df = pd.read_sql_query("""
    SELECT ROUND(CAST(SUM(product_price_£ * product_quantity) AS numeric),2) AS total_sales, month
    FROM orders_table
    JOIN dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
    JOIN dim_products ON dim_products.product_code = orders_table.product_code
    GROUP BY month
    ORDER BY total_sales DESC;
""", con=engine)
display(df)

,total_sales,month
0,673295.68,8
1,668041.45,1
2,657335.84,10
3,650321.43,5
4,645741.70,7
5,645463.00,3
6,635578.99,6
7,635329.09,12
8,633993.62,9
9,630757.08,11


## Task 4: How many sales are coming from online?

The company is looking to increase its online sales.
They want to know how many sales are happening online vs offline.
Calculate how many products were sold and the amount of sales made for online and offline purchases.
You should get the following information:
| numbers_of_sales | product_quantity_count  | location |
|------------------|-------------------------|----------|
|            26957 |                  107739 | Web      |
|            93166 |                  374047 | Offline  |

In [8]:
df = pd.read_sql_query("""
    SELECT 
        COUNT(*) AS number_of_sales, 
        SUM(orders_table.product_quantity) AS product_quantity_count, 
        CASE WHEN dim_store_details.store_type = 'Web Portal' THEN 'Web'
        ELSE 'Offline' END AS location
    FROM orders_table
    JOIN dim_store_details ON dim_store_details.store_code = orders_table.store_code
    GROUP BY location;
""", con=engine)
display(df)

,number_of_sales,product_quantity_count,location
0,93166,374047,Offline
1,26957,107739,Web


## Task 5: What percentage of sales come through each type of store?

The sales team wants to know which of the different store types is generated the most revenue so they know where to focus.
Find out the total and percentage of sales coming from each of the different store types.
The query should return:
| store_type  | total_sales | percentage_total(%) |
|-------------|-------------|---------------------|
| Local       |  3440896.52 |               44.87 |
| Web portal  |  1726547.05 |               22.44 |
| Super Store |  1224293.65 |               15.63 |
| Mall Kiosk  |   698791.61 |                8.96 |
| Outlet      |   631804.81 |                8.10 |


In [9]:
df = pd.read_sql_query("""
    WITH 
    sales_per_store AS (
    SELECT 
        store_type,
        ROUND((SUM(product_price_£ * product_quantity)::NUMERIC),2) AS total_sales
    FROM orders_table
    JOIN dim_store_details ON dim_store_details.store_code = orders_table.store_code
    JOIN dim_products ON dim_products.product_code = orders_table.product_code
    GROUP BY dim_store_details.store_type
    ),
    all_sales AS (
    SELECT SUM(total_sales) AS all_sales
    FROM sales_per_store
    )
    SELECT 
    store_type,
    total_sales, 
    ROUND((total_sales / all_sales) *100,2) AS percentage_total
    FROM sales_per_store, all_sales
    ORDER BY total_sales DESC;
""", con=engine)
display(df)

,store_type,total_sales,percentage_total
0,Local,3440896.52,44.56
1,Web Portal,1726547.05,22.36
2,Super Store,1224293.65,15.85
3,Mall Kiosk,698791.61,9.05
4,Outlet,631804.81,8.18


## Task 6: Which month in each year produced the highest cost of sales?

The company stakeholders want assurances that the company has been doing well recently.
Find which months in which years have had the most sales historically.
The query should return the following information:
| total_sales | year | month |
|-------------|------|-------|
|    27936.77 | 1994 |     3 |
|    27356.14 | 2019 |     1 |
|    27091.67 | 2009 |     8 |
|    26679.98 | 1997 |    11 |
|    26310.97 | 2018 |    12 |
|    26277.72 | 2019 |     8 |
|    26236.67 | 2017 |     9 |
|    25798.12 | 2010 |     5 |
|    25648.29 | 1996 |     8 |
|    25614.54 | 2000 |     1 |

In [10]:
df = pd.read_sql_query("""
    SELECT ROUND(CAST(SUM(product_price_£ * product_quantity) AS numeric),2) AS total_sales, year, month
    FROM orders_table
    JOIN dim_date_times ON dim_date_times.date_uuid = orders_table.date_uuid
    JOIN dim_products ON dim_products.product_code = orders_table.product_code
    GROUP BY year, month
    ORDER BY total_sales DESC
    LIMIT 10;
""", con=engine)
display(df)

,total_sales,year,month
0,27936.77,1994,3
1,27356.14,2019,1
2,27091.67,2009,8
3,26679.98,1997,11
4,26310.97,2018,12
5,26277.72,2019,8
6,26236.67,2017,9
7,25798.12,2010,5
8,25648.29,1996,8
9,25614.54,2000,1


## Task 7: What is our staff headcount?

The operations team would like to know the overall staff numbers in each location around the world. Perform a query to determine the staff numbers in each of the countries the company sells in.
The query should return the values:
| total_staff_numbers | country_code |
|---------------------|--------------|
|               13307 | GB           |
|                6123 | DE           |
|                1384 | US           |

In [11]:
df = pd.read_sql_query("""
    SELECT SUM(staff_numbers) AS total_staff_numbers, country_code
    FROM dim_store_details
    GROUP BY country_code
    ORDER BY total_staff_numbers DESC;
""", con=engine)
display(df)

,total_staff_numbers,country_code
0,13307,GB
1,6123,DE
2,1384,US


## Task 8: Which German store type is selling the most?
The sales team is looking to expand their territory in Germany. Determine which type of store is generating the most sales in Germany.
The query will return:
| total_sales  | store_type  | country_code |
|--------------|-------------|--------------|
|   198373.57  | Outlet      | DE           |
|   247634.20  | Mall Kiosk  | DE           |
|   384625.03  | Super Store | DE           |
|  1109909.59  | Local       | DE           |

In [12]:
df = pd.read_sql_query("""
    SELECT 
        ROUND(CAST(SUM(product_price_£ * product_quantity) AS numeric),2) AS total_sales,
        store_type,
        country_code
    FROM orders_table
    JOIN dim_store_details ON dim_store_details.store_code = orders_table.store_code
    JOIN dim_products ON dim_products.product_code = orders_table.product_code
    WHERE country_code IN ('DE')
    GROUP BY store_type, country_code
    ORDER BY total_sales;
""", con=engine)
display(df)

,total_sales,store_type,country_code
0,198373.57,Outlet,DE
1,247634.20,Mall Kiosk,DE
2,384625.03,Super Store,DE
3,1109909.59,Local,DE


## Task 9: How quickly is the company making sales?

Sales would like the get an accurate metric for how quickly the company is making sales.
Determine the average time taken between each sale grouped by year, the query should return the following information:

| year |                           actual_time_taken           |
|------|-------------------------------------------------------|
| 2013 | "hours": 2, "minutes": 17, "seconds": 12, "millise... |
| 1993 | "hours": 2, "minutes": 15, "seconds": 35, "millise... |
| 2002 | "hours": 2, "minutes": 13, "seconds": 50, "millise... | 
| 2022 | "hours": 2, "minutes": 13, "seconds": 6,  "millise... |
| 2008 | "hours": 2, "minutes": 13, "seconds": 2,  "millise... |

In [13]:
df = pd.read_sql_query("""
    WITH
    time_diff AS
        (SELECT year::numeric, month::numeric, day::numeric, timestamp::time, (LEAD(timestamp::time) OVER (ORDER BY year, month, day, timestamp::time) - timestamp::time) AS time_difference
        FROM dim_date_times
        ORDER BY year, month, day, timestamp),
    adj_time_diff AS
        (SELECT year, month, day, timestamp,
        CASE
            WHEN time_difference < '00:00:00' THEN time_difference + '24 hours'::interval
            ELSE time_difference
        END AS adjusted_value
        FROM time_diff)

    SELECT year,
        CONCAT(
            '"hours": ',        EXTRACT(hour FROM AVG(adjusted_value)),
            ', "minutes": ',      EXTRACT(minute FROM AVG(adjusted_value)),
            ', "seconds": ',      EXTRACT(second FROM AVG(adjusted_value)),
            ', "milliseconds": ', EXTRACT(milliseconds FROM AVG(adjusted_value))) AS actual_time_taken
    FROM adj_time_diff
    GROUP BY year
    ORDER BY AVG(adjusted_value) DESC
    LIMIT 5;
""", con=engine)
display(df)

,year,actual_time_taken
0,2013.0,"""hours"": 2, ""minutes"": 16, ""seconds"": 53.10258..."
1,1993.0,"""hours"": 2, ""minutes"": 15, ""seconds"": 40.12951..."
2,2002.0,"""hours"": 2, ""minutes"": 13, ""seconds"": 49.47822..."
3,2022.0,"""hours"": 2, ""minutes"": 13, ""seconds"": 2.169559..."
4,1995.0,"""hours"": 2, ""minutes"": 12, ""seconds"": 59.08451..."
